# Aim

Solve an LQR toy problem to see what to do

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import linalg as la
from scipy import stats as st

inv = np.linalg.inv
mvn = st.multivariate_normal
rms = lambda data: round(np.sqrt(np.mean((data - np.mean(data)) ** 2)), 4)

In [3]:
import sys
from os.path import dirname, abspath
sys.path.append(dirname(dirname(abspath("__file__"))))
from src import *

Running in simulation mode.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/adityasengupta/research/ao/set-tt-control/notebooks/data/bestflats/bestflat.npy'

In [ ]:
s = 1
m = 1
p = 1
nsteps = 10000

In [ ]:
lambdas = np.random.uniform(-1, 1, s)
lambdas = lambdas / ((1 + 1e-6) * np.max(np.abs(lambdas)))
M = np.random.randn(s,s)
A = np.linalg.inv(M) @ np.diag(lambdas) @ M
B = np.random.randn(s,m)
C = np.random.randn(p,s)
Q = np.eye(s)
R = np.eye(m)
W = 0.5 * np.eye(s)
V = 10 * np.eye(p)
process_dist = mvn(cov=W)
measure_dist = mvn(cov=V)
x = process_dist.rvs()

In [ ]:
kalman_cov = la.solve_discrete_are(A.T, C.T, W, V)
kalman_gain = kalman_cov @ C.T @ inv(C @ kalman_cov @ C.T + V)
lqr_cov = la.solve_discrete_are(A, B, Q, R)
lqr_gain = -inv(R + B.T @ lqr_cov @ B) @ B.T @ lqr_cov @ A

In [ ]:
states_un = np.zeros((nsteps, s))
states_un[0,:] = x
for i in range(1, nsteps):
    states_un[i,:] = A @ states_un[i-1,:] + process_dist.rvs()
    
rms(states_un)

12.204

In [ ]:
states = np.zeros((nsteps, s))
states_hat = np.zeros((nsteps, s))
states[0,:] = x
states_hat[0,:] = x
for i in range(1, nsteps):
    x = states[i-1,:]
    x_hat = states_hat[i-1,:]
    u = lqr_gain @ x_hat
    y = C @ (A @ x + B @ u) + measure_dist.rvs()
    innovation = y - C @ (A @ x_hat + B @ u)
    states_hat[i,:] = A @ x_hat + B @ u + kalman_gain @ innovation
    states[i,:] = states[i,:] + process_dist.rvs()
    
rms(states)

0.7003

In [ ]:
lqr_cov

array([[1.80409856]])